**Input:** 
- ACCEL-UKBB IDs

**Process:**
 - Attempt merging the leftover entries

**Output:**
 - Newly paired IDs

# Preparation (Execute all in this section!)

## Import libraries & set environment variables

In [1]:
import collections
import csv
from datetime import datetime
import os
import numpy as np
from pathlib import Path
import polars as pl
import re

# Draw flowcharts using Mermaid
import base64
from IPython.display import Image, display

def mm(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))

dir_home = Path(os.getcwd()).parent.parent
os.chdir(dir_home)
print("Current directory (check that it's your home directory):", os.getcwd())

Current directory (check that it's your home directory): J:\sugai\UKBiobank


In [4]:
mm("""
graph LR;
    classDef sourcedata fill:#FFFFFF
    classDef final fill:#BBBBBB
    
    New[New dataset: 671006] --> Pair[ID pairs];
    Old[Old dataset: 34134] --> Pair
    New --> Left1[Leftover IDs]
    Old --> Left2[Leftover IDs]
    Left1 --> Merge[Attempt merge]
    Left2 --> Merge
""")


In [2]:
# Input
DIR_SOURCE = os.path.join("data", "accel_ukbb", "merging")
FILE_OLD = os.path.join(DIR_SOURCE, "nonpaired_data_old.csv")
FILE_NEW = os.path.join(DIR_SOURCE, "nonpaired_data_new.csv")
FILE_IDS_ALREADY_PAIRED = os.path.join(DIR_SOURCE, "pair_ids.csv")
DIR_OUT = DIR_SOURCE

if not os.path.exists(DIR_OUT):
    os.makedirs(DIR_OUT)

In [3]:
def func_join_subset(df_old, df_new, cols_subset, cols_groupby):
    print(df_old.shape[0], "entries are found in the old dataset")
    print(df_new.shape[0], "entries are found in the new dataset")
    
    cols_subset_old = ["eid_old"] + cols_subset
    cols_subset_new = ["eid"] + cols_subset
    
    print(len(cols_subset), "Columns used for join:", cols_subset)
    # Extract the part used for join
    # Omit duplicates to prevent wrong join
    df_old_for_join = (df_old[cols_subset_old]
                        .unique(subset=cols_subset, keep="none"))
    df_new_for_join = (df_new[cols_subset_new]
                        .unique(subset=cols_subset, keep="none"))

    print(df_old_for_join.shape[0], "entries in the old dataset after omitting duplicates")
    print(df_new_for_join.shape[0], "entries in the new dataset after omitting duplicates")
    print()

    # Join and extract successful matches
    df_ids_joined = (df_new_for_join
                 .join(df_old_for_join, on=cols_subset, 
                         how="inner")
                 .drop_nulls()[["eid", "eid_old"]]
                  )
    print(df_ids_joined.shape[0], "entry pairs are successfully joined")
    # Extract leftover IDs from the join
    ids_leftover_old = sorted(list(
        set(df_old["eid_old"]) - set(df_ids_joined["eid_old"])
    ))
    ids_leftover_new = sorted(list(
        set(df_new["eid"]) - set(df_ids_joined["eid"])
    ))
    # Extract leftover entries
    df_leftover_old = df_old.filter(pl.col("eid_old").is_in(ids_leftover_old))
    df_leftover_new = df_new.filter(pl.col("eid").is_in(ids_leftover_new))
    print(df_leftover_old.shape[0], "entries in the old dataset remain unpaired")
    print(df_leftover_new.shape[0], "entries in the new dataset remain unpaired")
    
    # Extract only the entries existing in both datasets
    # Concatenate the specified columns into the "grouped" colunn
    print("Omit singletons")
    df_leftover_old_grouped = (df_leftover_old
                       .with_columns(
                           pl.concat_str(*cols_groupby).alias("grouped")
                       )
                      )

    df_leftover_new_grouped = (df_leftover_new
                       .with_columns(
                           pl.concat_str(*cols_groupby).alias("grouped")
                       )
                      )
    # Count
    df_leftover_old_count = df_leftover_old_grouped.groupby("grouped").agg(pl.count("grouped").alias("count"))
    df_leftover_new_count = df_leftover_new_grouped.groupby("grouped").agg(pl.count("grouped").alias("count"))
    # Extract the part common to the two datasets
    entry_exist_both = df_leftover_old_count.join(df_leftover_new_count, on="grouped", how="inner")["grouped"]
    entry_exist_both = list(entry_exist_both)
    df_leftover_old = (df_leftover_old_grouped
                        .filter(pl.col("grouped").is_in(entry_exist_both))
                        .drop("grouped")
                       )
    df_leftover_new = (df_leftover_new_grouped
                        .filter(pl.col("grouped").is_in(entry_exist_both))
                        .drop("grouped")
                       )
    print(df_leftover_old.shape[0], "entries in the old dataset remain unpaired")
    print(df_leftover_new.shape[0], "entries in the new dataset remain unpaired")
    df_leftover_all = (pl.concat([df_leftover_old, df_leftover_new]))
    return df_ids_joined, df_leftover_old, df_leftover_new, df_leftover_all

# Process

## Read the files and preprocess

In [ ]:
df_old = (pl.read_csv(FILE_OLD, separator=","))
df_new = (pl.read_csv(FILE_NEW, separator="\t",
                     infer_schema_length=0))

# Extract common columns
cols_old = df_old.columns
cols_new = df_new.columns
print(len(cols_old), "columns were found in the old dataset")
print(len(cols_new), "columns were found in the new dataset")
cols_common = sorted(list(
    set(cols_old) & set(cols_new)
))
print(len(cols_common), "columns were found in both datasets")
df_old = df_old[cols_common]
df_new = df_new[cols_common]

In [37]:
# Cast to string
for col_name in df_old.columns:
    df_old = df_old.with_columns(pl.col(col_name).cast(pl.Utf8))

# NA -> ""
for col_name in df_new.columns:
    df_new = df_new.with_columns(
        pl.col(col_name).str.replace("NA", "")
    )
    
# Change and add row
df_old = df_old.rename({"eid": "eid_old"})
df_old = df_old.with_columns(pl.lit(None).alias("eid").cast(pl.Utf8))
df_new = df_new.with_columns(pl.lit(None).alias("eid_old").cast(pl.Utf8))
# Make the orders of columns the same
df_new = df_new.select(df_old.columns)

# Omit complete duplicates
cols_except_id = [x for x in cols_common if not "eid" in x]
df_old = df_old.unique(subset=cols_except_id, keep="none")
df_new = df_new.unique(subset=cols_except_id, keep="none")

In [172]:
df_old

110005-0.0,110006-0.0,191-0.0,21000-0.0,21000-1.0,21000-2.0,21003-0.0,21003-1.0,21003-2.0,31-0.0,34-0.0,40000-0.0,40000-1.0,52-0.0,53-0.0,53-1.0,53-2.0,54-0.0,54-1.0,54-2.0,90001-0.0,90001-1.0,90001-2.0,90001-3.0,90001-4.0,90004-0.0,eid_old,eid
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
"""""","""""","""""","""1001""","""""","""""","""69""","""""","""""","""1""","""1941""","""""","""""","""2""","""2010-06-08""","""""","""""","""11014""","""""","""""","""""","""""","""""","""""","""""","""""","""1000548""",null
"""""","""""","""""","""1001""","""""","""""","""60""","""""","""""","""1""","""1947""","""2013-09-18""","""""","""6""","""2008-01-10""","""""","""""","""11006""","""""","""""","""""","""""","""""","""""","""""","""""","""1000967""",null
"""""","""""","""""","""1001""","""""","""""","""43""","""""","""53""","""1""","""1965""","""""","""""","""2""","""2008-11-06""","""""","""2019-01-26""","""11010""","""""","""11025""","""""","""""","""""","""""","""""","""""","""1001325""",null
"""""","""""","""""","""1001""","""""","""""","""59""","""""","""""","""0""","""1950""","""""","""""","""1""","""2009-10-23""","""""","""""","""11016""","""""","""""","""""","""""","""""","""""","""""","""""","""1002609""",null
"""2""","""""","""""","""1001""","""""","""1001""","""48""","""""","""56""","""0""","""1960""","""""","""""","""4""","""2008-07-23""","""""","""2016-10-22""","""11010""","""""","""11025""","""90001_0_0""","""""","""""","""""","""""","""90004_0_0""","""1003095""",null
"""""","""""","""""","""1001""","""1001""","""""","""63""","""67""","""""","""0""","""1945""","""""","""""","""6""","""2008-10-14""","""2013-01-05""","""""","""11008""","""11024""","""""","""""","""""","""""","""""","""""","""""","""1004539""",null
"""""","""""","""""","""1001""","""""","""""","""43""","""""","""""","""1""","""1965""","""""","""""","""1""","""2008-05-01""","""""","""""","""11010""","""""","""""","""""","""""","""""","""""","""""","""""","""1005207""",null
"""""","""""","""""","""1001""","""""","""""","""63""","""""","""""","""0""","""1944""","""""","""""","""9""","""2008-03-28""","""""","""""","""11004""","""""","""""","""""","""""","""""","""""","""""","""""","""1005642""",null
"""""","""""","""""","""1001""","""""","""""","""58""","""""","""69""","""1""","""1950""","""""","""""","""2""","""2008-05-13""","""""","""2019-03-08""","""11009""","""""","""11027""","""""","""""","""""","""""","""""","""""","""1006186""",null


limit_output extension: Maximum message size of 10000 exceeded with 16684 characters

In [204]:
df_new

110005-0.0,110006-0.0,191-0.0,21000-0.0,21000-1.0,21000-2.0,21003-0.0,21003-1.0,21003-2.0,31-0.0,34-0.0,40000-0.0,40000-1.0,52-0.0,53-0.0,53-1.0,53-2.0,54-0.0,54-1.0,54-2.0,90001-0.0,90001-1.0,90001-2.0,90001-3.0,90001-4.0,90004-0.0,eid_old,eid
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""""","""""","""""","""1001""","""""","""""","""59""","""""","""""","""0""","""1949""","""2010-07-07""","""""","""8""","""2008-10-27""","""""","""""","""11013""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1002493"""
"""""","""""","""""","""1001""","""""","""""","""70""","""""","""""","""0""","""1938""","""""","""""","""6""","""2008-08-13""","""""","""""","""11010""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1002974"""
"""""","""""","""""","""1001""","""""","""""","""68""","""""","""""","""1""","""1939""","""2011-01-11""","""""","""4""","""2007-12-15""","""""","""""","""11004""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1003797"""
"""""","""""","""""","""1001""","""""","""""","""51""","""""","""""","""1""","""1956""","""""","""""","""1""","""2007-12-14""","""""","""""","""11004""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1004555"""
"""""","""""","""""","""1001""","""""","""""","""64""","""""","""""","""1""","""1943""","""2019-04-10""","""""","""8""","""2008-03-27""","""""","""""","""11005""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1004631"""
"""""","""""","""""","""1001""","""""","""""","""51""","""""","""""","""0""","""1957""","""2012-09-14""","""""","""7""","""2009-06-13""","""""","""""","""11016""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1006189"""
"""""","""""","""""","""4001""","""""","""""","""68""","""""","""""","""0""","""1939""","""2010-11-24""","""""","""4""","""2007-05-03""","""""","""""","""11001""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1007687"""
"""""","""""","""""","""1001""","""""","""""","""46""","""""","""55""","""1""","""1963""","""""","""""","""5""","""2009-08-15""","""""","""2019-01-05""","""11017""","""""","""11027""","""""","""""","""""","""""","""""","""""",null,"""1008040"""
"""""","""""","""""","""1001""","""""","""""","""48""","""""","""""","""1""","""1959""","""""","""""","""9""","""2008-06-14""","""""","""""","""11009""","""""","""""","""""","""""","""""","""""","""""","""""",null,"""1009097"""


limit_output extension: Maximum message size of 10000 exceeded with 16628 characters

In [234]:
FILE_OUT = os.path.join(DIR_SOURCE, "nonpaired_leftover_data_old_0.csv")
df_old.write_csv(FILE_OUT)

FILE_OUT = os.path.join(DIR_SOURCE, "nonpaired_leftover_data_new_0.csv")
df_new.write_csv(FILE_OUT)

## Join, extract ID pairs and leftover entries

In [210]:
cols_subset = cols_except_id

cols_groupby = ["21000-0.0", # Ethnic background
                "31-0.0", # Sex
                "34-0.0", # Birth year
                "53-0.0", # Date of attending assessment centre, 1st
                "54-0.0", # Assessment centre
               ]

df_ids_joined_1, df_leftover_old_1, df_leftover_new_1, df_leftover_all_1 = func_join_subset(
    df_old, df_new, cols_subset, cols_groupby)

4703 entries are found in the old dataset
6916 entries are found in the new dataset
26 Columns used for join: ['110005-0.0', '110006-0.0', '191-0.0', '21000-0.0', '21000-1.0', '21000-2.0', '21003-0.0', '21003-1.0', '21003-2.0', '31-0.0', '34-0.0', '40000-0.0', '40000-1.0', '52-0.0', '53-0.0', '53-1.0', '53-2.0', '54-0.0', '54-1.0', '54-2.0', '90001-0.0', '90001-1.0', '90001-2.0', '90001-3.0', '90001-4.0', '90004-0.0']
4703 entries in the old dataset after omitting duplicates
6916 entries in the new dataset after omitting duplicates

4247 entry pairs are successfully joined
456 entries in the old dataset remain unpaired
2669 entries in the new dataset remain unpaired
Omit singletons
306 entries in the old dataset remain unpaired
308 entries in the new dataset remain unpaired


In [233]:
FILE_OUT = os.path.join(DIR_SOURCE, "nonpaired_leftover_data_old_1.csv")
df_leftover_old_1.write_csv(FILE_OUT)

FILE_OUT = os.path.join(DIR_SOURCE, "nonpaired_leftover_data_new_1.csv")
df_leftover_new_1.write_csv(FILE_OUT)

In [212]:
cols_subset = ["21000-0.0", # Ethnic background
                "31-0.0", # Sex
                "34-0.0", # Birth year
                "53-0.0", # Date of attending assessment centre, 1st
                "54-0.0", # Assessment centre
               ]

cols_groupby = ["21000-0.0", # Ethnic background
                "31-0.0", # Sex
                "34-0.0", # Birth year
                "53-0.0", # Date of attending assessment centre, 1st
                "54-0.0", # Assessment centre
               ]

df_ids_joined_2, df_leftover_old_2, df_leftover_new_2, df_leftover_all_2 = func_join_subset(
    df_leftover_old_1, df_leftover_new_1, cols_subset, cols_groupby)

306 entries are found in the old dataset
308 entries are found in the new dataset
5 Columns used for join: ['21000-0.0', '31-0.0', '34-0.0', '53-0.0', '54-0.0']
294 entries in the old dataset after omitting duplicates
293 entries in the new dataset after omitting duplicates

293 entry pairs are successfully joined
13 entries in the old dataset remain unpaired
15 entries in the new dataset remain unpaired
Omit singletons
13 entries in the old dataset remain unpaired
15 entries in the new dataset remain unpaired


In [215]:
cols_subset = ["53-0.0", # Date of attending assessment centre, 1st
               "53-1.0", # Date of attending assessment centre, 2nd
               "53-2.0", # Date of attending assessment centre, 3nd
                "54-0.0", # Assessment centre
               ]

cols_groupby = ["21000-0.0", # Ethnic background
                "31-0.0", # Sex
                "34-0.0", # Birth year
                "53-0.0", # Date of attending assessment centre, 1st
                "54-0.0", # Assessment centre
               ]

df_ids_joined_3, df_leftover_old_3, df_leftover_new_3, df_leftover_all_3 = func_join_subset(
    df_leftover_old_2, df_leftover_new_2, cols_subset, cols_groupby)

13 entries are found in the old dataset
15 entries are found in the new dataset
4 Columns used for join: ['53-0.0', '53-1.0', '53-2.0', '54-0.0']
13 entries in the old dataset after omitting duplicates
13 entries in the new dataset after omitting duplicates

8 entry pairs are successfully joined
5 entries in the old dataset remain unpaired
7 entries in the new dataset remain unpaired
Omit singletons
5 entries in the old dataset remain unpaired
7 entries in the new dataset remain unpaired


In [217]:
cols_subset = ["53-0.0", # Date of attending assessment centre, 1st
               "53-1.0", # Date of attending assessment centre, 2nd
               ]

cols_groupby = ["21000-0.0", # Ethnic background
                "31-0.0", # Sex
                "34-0.0", # Birth year
                "53-0.0", # Date of attending assessment centre, 1st
                "54-0.0", # Assessment centre
               ]

df_ids_joined_4, df_leftover_old_4, df_leftover_new_4, df_leftover_all_4 = func_join_subset(
    df_leftover_old_3, df_leftover_new_3, cols_subset, cols_groupby)

5 entries are found in the old dataset
7 entries are found in the new dataset
2 Columns used for join: ['53-0.0', '53-1.0']
5 entries in the old dataset after omitting duplicates
5 entries in the new dataset after omitting duplicates

5 entry pairs are successfully joined
0 entries in the old dataset remain unpaired
2 entries in the new dataset remain unpaired
Omit singletons
0 entries in the old dataset remain unpaired
0 entries in the new dataset remain unpaired


# Merge ID pair lists and export

In [223]:
df_ids_0 = pl.read_csv(FILE_IDS_ALREADY_PAIRED)
df_ids_0

eid_old,eid
i64,i64
1327806,1000010
2071371,1000028
3017169,1000034
2373493,1000045
3647458,1000052
1477405,1000069
3705388,1000076
3102569,1000087
3867761,1000091


In [231]:
cols_order = df_ids_0.columns

df_ids_all = (pl.concat([
    df_ids_0,
    df_ids_joined_1.with_columns(pl.col(cols_order).cast(pl.Int64))[cols_order],
    df_ids_joined_2.with_columns(pl.col(cols_order).cast(pl.Int64))[cols_order],
    df_ids_joined_3.with_columns(pl.col(cols_order).cast(pl.Int64))[cols_order],
    df_ids_joined_4.with_columns(pl.col(cols_order).cast(pl.Int64))[cols_order],
])
              .sort(by="eid")
             )


FILE_OUT = os.path.join(DIR_SOURCE, "pair_ids_20230518.csv")
df_ids_all.write_csv(FILE_OUT)

df_ids_all

eid_old,eid
i64,i64
1327806,1000010
2071371,1000028
3017169,1000034
2373493,1000045
3647458,1000052
1477405,1000069
3705388,1000076
3102569,1000087
3867761,1000091


# Final leftover IDs

In [3]:
df_ids_all = pl.read_csv(os.path.join(DIR_SOURCE, "pair_ids_20230518.csv"))
df_ids_all

eid_old,eid
i64,i64
1327806,1000010
2071371,1000028
3017169,1000034
2373493,1000045
3647458,1000052
1477405,1000069
3705388,1000076
3102569,1000087
3867761,1000091
